In [ ]:
### INSTALLATION OF REQUIREMENTS ###

# Installs the required dependancies as colab starts with a clean slate
# Diffusers are a text-to-image diffusion model for generating the realistic
#   synthetic dataset to be used by the Machine Learning
!pip install --upgrade diffusers[torch]

# Transformers work with Diffusers to generate better photos for the Synthetic Dataset
!pip install transformers

In [ ]:
### CREATE IMAGE GENERATION PIPELINE ###

# Imports
from diffusers import DiffusionPipeline
import torch

# Creates the object for generation from the pre-trained diffusion model
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
pipeline.to("cuda")

In [ ]:
### GENERATE IMAGES ###

# Imports
import random
import os

# Used for testing the quality of the photos to show
import matplotlib.pyplot as plt

# Making the directories for the specific folders for each emotion
os.makedirs('/content/faces/happy', exist_ok=True)
os.makedirs('/content/faces/sad', exist_ok=True)
os.makedirs('/content/faces/angry', exist_ok=True)
os.makedirs('/content/faces/surprised', exist_ok=True)
os.makedirs('/content/faces/neutral', exist_ok=True)

# Ethnicities and genders
ethnicities = ['a latino', 'a white', 'a black', 'a middle eastern', 'an indian', 'an asian']
genders = ['male', 'female']

# Prompts for the emotions and explaining the emotions using a dictionary with
#   key: value pair
emotion_prompts = {'happy': 'smiling',
                   'sad': 'frowning, sad face expression, crying',
                   'surprised': 'surprised, opened mouth, raised eyebrows',
                   'angry': 'angry',
                   'neutral': 'emotionless, blank expression'}

# Get 250 pictures of each emotion in emotion_prompts
for j in range(250):

  # Runs each emotion that is set as the key (happy->sad->surprised->angry->neutral) then restarts over
  #   for the next interation
  for emotion in emotion_prompts.keys():

    # Gets the value from the emotion_prompts dictionary and places it into emotion_prompt
    emotion_prompt = emotion_prompts[emotion]

    # Randomly choose the ethnicity and the gender of the person being generated
    ethnicity = random.choice(ethnicities)
    gender = random.choice(genders)

    # The posotive prompt to generate pictures by running through all of the ethnicities, genders, and emotions making ethinicity and gender randomly chosen
    #   Must be very specific as the model generator (Stable Diffusion) can generate any image
    prompt = 'Medium-shot portrait of {} {}, {}, front view, looking at the camera, color photography, '.format(ethnicity, gender, emotion_prompt) + \
            'photorealistic, hyperrealistic, realistic, incredibly detailed, crisp focus, digital art, depth of field, 50mm, 8k'

    # The negative prompt for pipeline() to try and use as guidelines for what not to generate
    negative_prompt = '3d, cartoon, anime, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ' + \
                      '((grayscale)) Low Quality, Worst Quality, plastic, fake, disfigured, deformed, blurry, bad anatomy, blurred, watermark, grainy, signature'

    # Placing the photo into img and saving it into 'content/faces/(emotion) and filling the number
    #   with what iteration j is on with leading zeros to fill 4 digits
    img = pipeline(prompt, negative_prompt=negative_prompt).images[0]
    img.save('/content/faces/{}/{}.png'.format(emotion, str(j).zfill(4)))


In [ ]:
# Zipping the folders into faces.zip
!zip -r faces.zip /content/faces

In [ ]:
# Mounting Google Drive so that the zipped folders can be saved directly to Google Drive
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
# Takes the zipped folder and places it into the designated folder inside of my Google Drive
!scp '/content/faces.zip' '/content/gdrive/My Drive/SyntheticDatasetFaceGeneration/faces.zip'